In [2]:
import pandas as pd
import json
from glob import glob

In [36]:
# Replace old or outdated csv, or relocate to different directory
sensor_in_path = glob("*.csv")[0]
sensor_csv = pd.read_csv(sensor_in_path)

In [37]:
# Verify that correct csv is loaded
sensor_csv.head()

,Sensors,Subteam
0,FL_VSS,Suspension-Powertrain-Chassis
1,FR_VSS,Suspension-Powertrain-Chassis
2,BL_VSS,Suspension-Powertrain-Chassis
3,BR_VSS,Suspension-Powertrain-Chassis
4,FL_BRK_TMP,Suspension


In [38]:
def parse_subteam_name(team_str: str) -> tuple:
    """
    Splits a subteam group with delimiter '-'.
    Groups containing only sentinel value 'nu' are considered empty.
    """
    if team_str == 'nu':
        return ()
    return tuple(team_str.strip().split('-'))

In [39]:
# Parse subteams by sensor
sensor_csv["Parsed"] = sensor_csv["Subteam"].apply(parse_subteam_name)
sensor_csv.head()

,Sensors,Subteam,Parsed
0,FL_VSS,Suspension-Powertrain-Chassis,"(Suspension, Powertrain, Chassis)"
1,FR_VSS,Suspension-Powertrain-Chassis,"(Suspension, Powertrain, Chassis)"
2,BL_VSS,Suspension-Powertrain-Chassis,"(Suspension, Powertrain, Chassis)"
3,BR_VSS,Suspension-Powertrain-Chassis,"(Suspension, Powertrain, Chassis)"
4,FL_BRK_TMP,Suspension,"(Suspension,)"


In [40]:
# Convert to dict by subteam
by_subteam = {}
for row in sensor_csv.itertuples():
    teams = row.Parsed
    for team in teams:
        if team not in by_subteam.keys():
            by_subteam[team] = [row.Sensors]
        else:
            by_subteam[team].append(row.Sensors)


In [41]:
by_subteam

{'Suspension': ['FL_VSS',
  'FR_VSS',
  'BL_VSS',
  'BR_VSS',
  'FL_BRK_TMP',
  'FR_BRK_TMP',
  'BL_BRK_TMP',
  'BR_BRK_TMP',
  'FL_SUS_POT',
  'FR_SUS_POT',
  'BR_SUS_POT',
  'BL_SUS_POT',
  'F_BRK_PRES',
  'B_BRK_PRES',
  'STEER_ANG',
  'ACCELX',
  'ACCELY',
  'ACCELZ',
  'BATT_VLT'],
 'Powertrain': ['FL_VSS',
  'FR_VSS',
  'BL_VSS',
  'BR_VSS',
  'TPS',
  'OIL_PRES',
  'OIL_TEMP',
  'COOL_TEMP',
  'MAP',
  'MAT',
  'NEUT',
  'LAMBDA1',
  'LAMBDA2',
  'BATT_VLT',
  'RPMS'],
 'Chassis': ['FL_VSS',
  'FR_VSS',
  'BL_VSS',
  'BR_VSS',
  'ACCELX',
  'ACCELY',
  'ACCELZ',
  'BATT_VLT'],
 'Frame': ['ACCELX', 'ACCELY', 'ACCELZ', 'BATT_VLT'],
 'Aerodynamics': ['STRAIN1',
  'STRAIN2',
  'STRAIN3',
  'STRAIN4',
  'PTUBE1',
  'PTUBE2',
  'PTUBE3',
  'PTUBE4',
  'PTUBE5',
  'PTUBE6',
  'PTUBE7',
  'PTUBE8',
  'PTUBE9',
  'PTUBE10',
  'PTUBE11',
  'PTUBE12',
  'BATT_VLT'],
 'Electronics': ['DAQTEMP', 'BATT_VLT'],
 'EV': ['BATT_VLT']}

In [42]:
# Store in .json file
sensor_out_path = "sensors_by_subteam.json"
open(sensor_out_path, 'w').write(json.dumps(by_subteam))

820